<a href="https://colab.research.google.com/github/wjunwei2001/TailorMade/blob/main/tiktok_llm_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate
# !pip install InstructorEmbedding
!pip install -U sentence-transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 8.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import re
import spacy
from nltk.corpus import stopwords
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
import torch
import sentence_transformers
import accelerate
from sentence_transformers import SentenceTransformer, losses
from sentence_transformers.losses import ContrastiveTensionDataLoader
from datasets import Dataset

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# Import SOTA text embedding model bsaed on the universally recognised Massive Text Embedding Benchmark (MTEB)

# model = SentenceTransformer('all-MiniLM-L6-v2')
# model = SentenceTransformer('EZlee/e-commerce-bert-base-multilingual-cased')
# model = SentenceTransformer("Salesforce/SFR-Embedding-2_R")
# model = SentenceTransformer('Alibaba-NLP/gte-large-en-v1.5', trust_remote_code=True)
# dimensions = 512
# model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1", truncate_dim=dimensions)
# model = SentenceTransformer('Alibaba-NLP/gte-base-en-v1.5', trust_remote_code=True)
model = SentenceTransformer("avsolatorio/GIST-Embedding-v0", revision=None)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/117k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

## LLM Finetuning (Self supervised)

In [ ]:
df = pd.read_csv('amazon_data.csv')

<ipython-input-3-ef29398646c6>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('amazon_data.csv')


In [ ]:
df['description'].replace(np.nan, '', inplace=True)
# df['description'] = df['description'].apply(''.join)

In [ ]:
df.head(5)

,title,original_price,sale_price,discount,rating,review_count,main_category,sub_category_1,sub_category_2,rankings,description,insights,purchase_cnt_prev_month,store_name,supplier,is_available,url
0,cycling sunglasses polarized men women sport s...,NaN,$7.99,NaN,3.3,3,Sports and Outdoors,Sports & Outdoors,Cycling Glasses & Goggles,"#52,093 in Sports & Outdoors (See Top 100 in S...",['【Flexible and Frame-less Design Frame】: The ...,NaN,NaN,ZJMZZM,ZJMZZM,In Stock,/sspa/click?ie=UTF8&spc=MToyMTgzOTMwMDMwOTIxMD...
1,"polarized sport sunglasses for men women, idea...",NaN,$19.99,NaN,4.5,38,Sports and Outdoors,Sports & Outdoors,Cycling Glasses & Goggles,"#270,555 in Sports & Outdoors (See Top 100 in ...",,NaN,NaN,ACBLUCE Store,ACBLUCE,In Stock,/sspa/click?ie=UTF8&spc=MToyMTgzOTMwMDMwOTIxMD...
2,"professional cycling glasses for men women, sp...",NaN,$8.49,NaN,4.6,54,Sports and Outdoors,Sports & Outdoors,Cycling Glasses & Goggles,"#2,851 in Sports & Outdoors (See Top 100 in Sp...",,"['Appearance_POSITIVE', 'Value_POSITIVE', 'Dar...",1K+ bought in past month,ROIBEAO,Roibeao Direct,In Stock,/sspa/click?ie=UTF8&spc=MToyMTgzOTMwMDMwOTIxMD...
3,rockbros bike bicycle cargo rack rear bike rac...,$43.99,$43.99,NaN,4.3,"2,803",Sports and Outdoors,Sports & Outdoors,Bike Cargo Racks,"#11,213 in Sports & Outdoors (See Top 100 in S...",['QUICK RELEASE CLAMP for easily mount on the ...,"['Quality_POSITIVE', 'Performance_POSITIVE', '...",700+ bought in past month,ROCKBROS Store,ROCKBROS LLC,In Stock,/RockBros-Bicycle-110-165-Capacity-Mountain/dp...
4,boao 20 pieces presta valve core bike replacem...,NaN,$9.99,NaN,4.6,832,Sports and Outdoors,Sports & Outdoors,Bike Tubes,"#9,574 in Sports & Outdoors (See Top 100 in Sp...",['Package include: 20 pieces presta valve core...,"['Quality_POSITIVE', 'Performance_POSITIVE', '...",600+ bought in past month,Boao Store,Hiroda US,In Stock,/Pieces-Replacement-Tubeless-Vittoria-Continen...


In [ ]:
df['main_category'] = df['main_category'].str.lower()
df['main_category'].value_counts()

main_category
baby products               5846
computers                   4363
home and kitchen            4230
electronics                 3854
boy's fashion               3619
mens's fashion              3534
beauty and personal care    3308
girl's fashion              3165
sports and outdoors         2853
toys and games              2584
women's fashion             2399
main_category                  2
Name: count, dtype: int64

In [ ]:
df['description'][0]

"['【Flexible and Frame-less Design Frame】: The newest Tr90 frame-less design for cycling glasses which make cycling sunglasses becoming more stylish and lightweight. The glasses leg is made of rubber which is flexible and comfortable.', '【UV400 Polarized Lenses】These Polarized Sport Sunglasses are took premium polarized lenses which are 100% UVA and UVB filtering, reduce reflections and glare for a clearer view. So your child can perform to the best of their ability in the Contest.', '【Ventilation Structure】: Our Sports Sunglasses have an excellent ventilation system with vents on the lenses and legs. This ventilation system not only prevents the lenses from fogging up, but also makes them more breathable when worn.', '【Designed for Sports】The Youth Sport Sunglasses are ideal for outdoor sports, such as baseball, softball, climbing, driving, fishing, golf, hiking, running, skiing, cycling and so on.', '【Anti Slip Design】: The legs of the polarized sunglasses are made of non-slip rubber

In [ ]:
df['ecommerce_text'] = df['title'] + ' ' + df['description']

In [ ]:
# Convert text to lowercase
df['ecommerce_text'] = df['ecommerce_text'].str.lower()

# Remove special characters and numbers
df['ecommerce_text'] = df['ecommerce_text'].apply(lambda x: re.sub('[^a-z\s]', '', str(x)))

In [ ]:
df['ecommerce_text'][5]

'air pump replacement nozzles  pack inflatable pump nozzles head air pump inflator adaptor accessory for partial air bed air mattress etc  sizes it respectively in small medium and large size mouth easily meeting your needs to inflatable things of different mouth sizes made of quality plastic durable for long time using and lightweight for carrying in your travel bag and family toolbox choose the nozzle of proper size accordingly and install it on the air pump and then inflate the target object the small size one fit for small toys medium and large one fits for swimming pool boat inflatable bed this nozzles head is suitable for different kinds of pumps helping you to inflate swimming rings air bed air mattress inflatable pool sofa and more a good tool accessory for outdoor activities'

In [ ]:
new_df = df.groupby('main_category').apply(lambda x: x.sample(min(len(x), 1000))).reset_index(drop=True)

# Verify the counts in the new DataFrame
print(new_df['main_category'].value_counts())

main_category
baby products               1000
beauty and personal care    1000
boy's fashion               1000
computers                   1000
electronics                 1000
girl's fashion              1000
home and kitchen            1000
mens's fashion              1000
toys and games              1000
sports and outdoors         1000
women's fashion             1000
main_category                  2
Name: count, dtype: int64


In [ ]:
train_examples = new_df['ecommerce_text'].tolist()
train_examples

['baby play mat tummy time mat with exclusive foldable design stagebased skill development activity gym machine washable baby play mats for floor with gift package zebra exclusive foldable design lil pengyu baby play mat with exclusive foldable design its easy transforms into a handy bag for storing small toys its the perfect companion for camping and everyday play practicality meets convenience gorgeous gift packaging present your little one with a stunning surprise lil pengyu tummy time mats for babies  months comes in a beautifully designed gift box making it the perfect perfect gift for any occasion give the gift of joy and imagination luxurious comfort crafted with  polyester lil pengyu baby play mats for floor features a front side made of premium silklike fabric that pampers your babys sensitive skin the plush toy attachments are crafted from soft and cuddly crystal velvet ensuring hours of irresistible playtime the perfect blend of comfort and entertainment stagebased motor ski

In [ ]:
train_dataloader = ContrastiveTensionDataLoader(train_examples, batch_size=8, pos_neg_ratio=8)
train_loss = losses.ContrastiveTensionLoss(model=model)

model.fit(
    [(train_dataloader, train_loss)],
    epochs=5,
)

Step,Training Loss
500,7.440300
1000,4.853600
1500,3.864600
2000,3.887700
2500,3.878900
3000,3.805200
3500,3.838500


In [ ]:
!pip install huggingface_hub

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write)

In [ ]:
model.push_to_hub("ecommerce_text_embedding")

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

'https://huggingface.co/wjunwei/ecommerce_text_embedding/commit/dc4821e729ac085fbb72934fcaf5bc7e78a9e166'

In [ ]:
ecommerce_model = SentenceTransformer("wjunwei/ecommerce_text_embedding")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/41.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

## Using the model for generating similarity score

In [ ]:
# Do the same text processing for inference
df['title'] = df['title'].str.lower()
df['title'] = df['title'].apply(lambda x: re.sub('[^a-z\s]', '', str(x)))

df['product_embedding'] = df['title'].apply(lambda x: ecommerce_model.encode(x))

In [ ]:
df['title'][10523]

'vanten kitchen sink caddy sponge holder sink organizer sink tray drainer rack soap dish dispenser brush holder storage accessories countertop or adhesive'

In [ ]:
user_input = "friend"
user_input2 = "housewife"

In [ ]:
user_input_embedding = model.encode(user_input)
user_input_embedding2 = model.encode(user_input2)

In [ ]:
def calculate_similarity(embedding1, embedding2):
    return cosine_similarity([embedding1], [embedding2])

result1 = calculate_similarity(user_input_embedding, df['product_embedding'][10523])
result2 = calculate_similarity(user_input_embedding2, df['product_embedding'][10523])

print(result1)
print(result2)

[[-0.6465126]]
[[-0.12326902]]
